# Simple Discrete-Time, Finite Time Horizon, Linear Quadratic Regulator (LQR) Implementation

In [1]:
# import statements
import numpy as np

In [21]:
# LQR routine

def lqr(A, B, Q, R, Q_f, N):
    """
    A, B      : state dynamics matrices (for x_t+1 = Ax_t + Bu_t)
    Q, R, Q_f : state and input weight matrices for function to be minimized
    N         : length of finite time horizon
    """
    
    # lists for P and K where element t represents P_t and K_t, respectively
    P = []
    K = []
    
    # P_N = Q_f
    P.insert(0, Q_f)
    
    # K_N = 0
    K.insert(0, 0)
    
    # work backwards from N-1 to 0
    for i in range(N-1, -1, -1):
        # calculate K_t
        temp1 = np.linalg.inv(R+np.dot(B.T, np.dot(P[0], B)))
        temp2 = np.dot(B.T, np.dot(P[0], A))
        K_t = np.dot(temp1, temp2)
        
        # calculate P_t
        AplusBK = A + np.dot(B, K_t)
        P_t = Q + np.dot(K_t.T, np.dot(R, K_t)) + np.dot(AplusBK.T, np.dot(P[0], AplusBK))
        
        # add our calculations to the beginning of the lists (since we are working backwards)
        K.insert(0, K_t)
        P.insert(0, P_t)
    
    return P, K

In [22]:
# routine to determine optimal cost-to-go and optimal control given an initial state x_0

def optimalCostAndControl(x_0, P, K, A, B, C):
    """
    x_0     : the initial state
    P, K    : matrices used to update optimal cost and control, respectively
    A, B, C : state dynamics matrices (for x_t+1 = Ax_t + Bu_t, y = Cx_t)
    """
    
    # lists for optimal cost and control where element t represents J_t and u_t, respectively
    J = []
    U = []
    
    # lists for state and output where element t represents x_t and y_t, respectively
    X = []
    Y = []
    
    # setting x_0
    X.append(x_0)
    
    # since P also contains P_N, we need to subtract one to get range [0, N-1]
    for i in range((len(P) - 1)):
        # calculate J_i and u_i
        J.append(np.dot(X[i].T, np.dot(P[i], X[i])))
        U.append(-1 * np.dot(K[i], X[i]))
        
        # calculate x_i+1 and y_i
        X.append(np.dot(A, X[i]) + np.dot(B, U[i]))
        Y.append(np.dot(C, X[i]))
    
    return J, U, X, Y

In [23]:
# example using routines written above where we want to control output instead of state

A = np.matrix([[1, 1], [0, 1]])
B = np.matrix([[0], [1]])
C = np.matrix([[1, 0]])

x_0 = np.matrix([[1], [0]])
N = 20

In [24]:
# case 1: TODO
p_q = 1
Q = p_q * np.identity(2)
Q_new = np.dot(C.T, np.dot(Q, C))
Q_f = np.zeros((2, 2))

R = 1

P, K = lqr(A, B, Q_new, R, Q_f, N)
J, U, X, Y = optimalCostAndControl(x_0, P, K, A, B, C)

ValueError: shapes (2,2) and (1,2) not aligned: 2 (dim 1) != 1 (dim 0)